一個模型裡可能有很多超參數，給定每個超參數一個範圍，我們便可以暴力破解，在所有排列組合中找到最好的組合。這稱為$\textbf{網格搜尋}$。我們一樣用乳癌數據集來測試，並用網格搜尋來尋找支援向量機實作時的最佳超參數。

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

df = pd.read_csv('wdbc.data', header=None) # 讀取乳癌數據集

# 類別mapping: M=1, B=0
X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_
le.transform(['M', 'B'])

# 以8:2的比例製造測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1) 

利用scikit-learn裡面的GridSearchCV來實作

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=1))

# 超參數挑選範圍
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] 
param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

# 使用網格搜尋法
gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

# 輸出結果
print(gs.best_score_) # 最佳的正確率
print(gs.best_params_) # 最佳參數組合

0.9846153846153847
{'svc__C': 100.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}


我們甚至可以直接把最佳的參數打包起來用。

In [3]:
clf = gs.best_estimator_ # 取得剛剛算出的最佳參數
clf.fit(X_train, y_train) # 用這個參數train出model
print('Test accuracy: %.3f' % clf.score(X_test, y_test)) # 對測試集進行預測

Test accuracy: 0.974


$\textbf{巢狀交叉驗證選擇演算法}$是一種可以幫助我們挑選機器學習演算法的方法。數學證明，這種演算法對測試集的真實錯誤估計幾乎沒有偏誤(就是正確率很好的意思)。

其作法是先將訓練集平分為k折(稱為外折)，取其中一折作為測試集，剩下k-1折結合起來平分成兩半(稱為內折)，一半作為training fold，一半作為validation fold來選擇模型。訓練完畢，便用一開始分出的測試集測試。我們通常取外折為5折，內折為2折，此稱為$\textbf{5x2 cross validation}$。

我們來比較看看支援向量機和決策樹，誰最適合拿來就乳癌數據集進行預測。

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# 支援向量機
gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2)
scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print('SVM CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

# 決策樹
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
                  scoring='accuracy',
                  cv=2)

scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print('DecisionTree CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

SVM CV accuracy: 0.974 +/- 0.015
DecisionTree CV accuracy: 0.934 +/- 0.016


由於SVM給了比較好的正確率，我們應該挑選SVM。